# Demonstration RAG-Eval


Quick test: see if things work as planned

In [1]:
from rag_eval.protocol import LLMClient
from rag_eval.client import AzureClient
from rag_eval.components import ClaimExtractor, EntailmentJudge
from rag_eval.data_models import EvalContainer, ClaimVerdict
from rag_eval.evaluators import AnswerPrecisionEvaluator, RetrievalPrecisionEvaluator
from rag_eval.coordinator import RAGEvaluator
from rag_eval.helpers import _safe_json
import json
from data.rag_eval_samples import test_examples
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

c:\Users\ANDERBE\AI-Agent-POC-git\AI-Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

api_key = os.getenv("OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
base_url = os.getenv("OPENAI_CHATCOMPLETIONS_URL")
model = os.getenv("OPENAI_DEPLOYMENT")
password = os.getenv("CORRECT_PASSWORD")

llm = AzureClient(
    api_key=api_key,
    api_version=api_version,
    base_url=base_url,
    model=model
)


In [ ]:
coord = RAGEvaluator(llm_extractor=llm, llm_judge=llm)
data = test_examples[0:2]


for i, ex in enumerate(data, start=1):
    print(f"EXAMPLE {i}")
    results = coord.evaluate_all(ex)
    print(json.dumps(results, indent=2))

EXAMPLE 1


KeyboardInterrupt: 

# Test Agent

Check if agent module actually works.

In [3]:
# force reload of agent-script after changes 
import importlib, inspect, agent.main as main

#print("Module file:", main.__file__)      
main = importlib.reload(main)               # force reload the module
#print("Reloaded file:", main.__file__)

# show the exact source Python is executing
#print(inspect.getsource(main.food_agent))

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [5]:
# load agent and pipeline for indexing docs
import agent.main as main
from rag.indexing import pdf_files, index_pipeline

In [6]:
index_pipeline.run({"converter": {"sources": pdf_files}})

incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
Batches: 100%|██████████| 2/2 [00:00<00:00,  2.55it/s]


{'writer': {'documents_written': 51}}

In [5]:
main.food_agent(user_input="hi")


▶️ Step 1: thinking…
🍕 Fooder: Hello! I'm Fooder, your food ordering agent. 🍔🍕🥤

Would you like to see the available food options or place an order today? Let me know how I can help you!


## RAG-Loop
Loop which passes questions to agent and extracts generated answers alongside retrieved contexts (plain text). Combines them with ground truth and query to form `EvalContainer`-object.

In [6]:
# quick test of my query_gt_extractor function
from src.rag_eval.helpers import query_gt_extractor

queries, gt = query_gt_extractor(path="data/agent_eval_questions.json", tokenize=False)
print(gt)


['2010', 'La cucina povera', 'Auguste\u202fEscoffier', 'Jerry\u202fThomas', '1920\u202f–\u202f1933', 'Orecchiette', 'Tarte\u202fTatin', '8,50\u202f€', 'Horchata', 'Secondo', 'Béchamel', 'Reis', 'Mezcal', 'Churros', '2,00\u202f€', 'Risotto\u202fal\u202fTartufo\u202fNero\u202fdel\u202fMastino', 'Mole', 'Camembert', 'Tiramisù\u202fClassico\u202f&\u202fCoda\u202fdi\u202fVolpe', 'Rindfleisch', 'Madrid', 'R', '1969', '7', 'Ernest\xa0Hemingway', 'Mais, Bohnen, Chili, Kürbis, Kakao; UNESCO‑Jahr\xa02010', 'Mexikanische Küche\xa02010; Gastronomisches Mahl der Franzosen\xa02010', 'Parmigiano\u202fReggiano, Mozzarella\u202fdi\u202fBufala; Camembert, Roquefort', 'Reis (Risotto) für Norditalien; Olivenöl für die Provence', 'Prosecco Extra Dry del Cucciolo\u202f–\u202f50,00\u202f€; Vin Rouge Français\u202f(0,75\u202fl)\u202f–\u202f18,00\u202f€', 'Martini, Manhattan, Daiquiri; Negroni', 'Elote – gegrillter Mais mit Limette und Käse; Caprese (Mozzarella, Tomate, Basilikum)', 'Béchamel; Filetto\u202fdi\

This code cell creates a RAG-loop for the agent. It:  
- feeds the questions from the test battery to the agent  
- extracts the generated answer and retrieved context from the RAG-call
- returns a list of `EvalContainer`-objects storing question, ground truth, generated answer and retrieved context

It thus lays the foundation for the `RAGEevaluator`. 

In [8]:
from src.rag_eval.helpers import query_gt_extractor
import agent.main as main
from rag_eval.data_models import EvalContainer

def rag_loop_agent() -> list[EvalContainer]:
    queries, gts = query_gt_extractor(path="data/agent_eval_questions.json", tokenize=False) # get questions and ground truth
    for (query, gt) in zip(queries, gts):
        result = main.food_agent(user_input=query)
        main.food_agent(user_input="quit")

        # TODO: extract generated answers, gt, question, context and store in EvalContainer
    return result




In [9]:
result = rag_loop_agent()


▶️ Step 1: thinking…
🍕 Fooder: Die mexikanische Küche wurde im Jahr 2010 von der UNESCO als immaterielles Weltkulturerbe anerkannt.
Goodbye 🍕

▶️ Step 1: thinking…
🍕 Fooder: Der italienische Begriff für die „Küche der armen Leute“, aus der viele klassische Gerichte stammen, ist cucina povera.
Goodbye 🍕

▶️ Step 1: thinking…
🍕 Fooder: Der französische Koch Auguste Escoffier brachte im 19. Jahrhundert Ordnung in die Küchenorganisation und schrieb das Werk «Le Guide Culinaire».
Goodbye 🍕

▶️ Step 1: thinking…
Invoked agentic RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 72.86it/s]


Goodbye 🍕

▶️ Step 1: thinking…
Invoked agentic RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.40it/s]


Goodbye 🍕

▶️ Step 1: thinking…
Invoked agentic RAG!


...

Goodbye 🍕

▶️ Step 1: thinking…
Invoked agentic RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 102.94it/s]


Goodbye 🍕

▶️ Step 1: thinking…
Invoked agentic RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.02it/s]


Goodbye 🍕


## Eval Loop
Extract individual queries from `EvalContainer-object` like so: 
```
container = test_examples[1]
print(container.query)
print(container)
```

## Ordinary Agent Test

In [13]:
# force reload of agent-script after changes 
import importlib, inspect, agent.backup.backup_main as backup

#print("Module file:", main.__file__)      
backup = importlib.reload(backup)               # force reload the module

In [ ]:
import agent.backup.backup_main as backup
backup.food_agent()


▶️ Step 1: thinking…
TOOL CALL ▶ execute_plain_rag {"question":"Where can I get tacos?"}
Invoked plain RAG!


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.05it/s]



▶️ Step 2: thinking…
🍕 Fooder: You can get tacos at "Tönis Tacos." Here are a few taco options from their menu:

- Carne Asada Taco (beef, onions, cilantro) - 3,50 €
- Pollo Taco (grilled chicken, pico de gallo) - 3,50 €
- Veggie Taco (grilled vegetables, avocado) - 3,00 €
- Fish Taco (fried fish, coleslaw) - 4,00 €

If you’d like to order, let me know which tacos you want!
🍕 Fooder: Goodbye 🍕


: 